** Summary **
This dataset (ml-latest) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. 

It contains 27753444 ratings and 1108997 tag applications across 58098 movies. 

These data were created by 283228 users between January 09, 1995 and September 26, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 1 movies. 

No demographic information is included. 

Each user is represented by an id, and no other information is provided.

The data are contained in the files 

    genome-scores.csv, 
    genome-tags.csv, 
    links.csv, 
    movies.csv, 
    ratings.csv and 
    tags.csv. 
    
More details about the contents and use of all these files follows.

This and other GroupLens data sets are publicly available for download at http://grouplens.org/datasets/.

# Ratings Data File Structure (ratings.csv)
All ratings are contained in the file ratings.csv. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

    userId,
    movieId,
    rating,
    timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

# Tags Data File Structure (tags.csv)
All tags are contained in the file tags.csv. Each line of this file after the header row represents one tag applied to one movie by one user, and has the following format:

    userId, 
    movieId, 
    tag, 
    timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Tags are user-generated metadata about movies. Each tag is typically a single word or short phrase. The meaning, value, and purpose of a particular tag is determined by each user.

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

# Movies Data File Structure (movies.csv)
Movie information is contained in the file movies.csv. Each line of this file after the header row represents one movie, and has the following format:

    movieId,
    title,
    genres
    
Movie titles are entered manually or imported from https://www.themoviedb.org/, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.

Genres are a pipe-separated list, and are selected from the following:

    Action
    Adventure
    Animation
    Children's
    Comedy
    Crime
    Documentary
    Drama
    Fantasy
    Film-Noir
    Horror
    Musical
    Mystery
    Romance
    Sci-Fi
    Thriller
    War
    Western
    (no genres listed)

In [1]:
import os
import time

# data science imports
import math
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [11]:
# environment variables 
env_var = os.environ 
env_var['DATA_PATH']

'E:\\MYLEARN\\2-ANALYTICS-DataScience\\datasets'

# 1. Load Data

In [2]:
# data location
#data_path = os.path.join(os.environ['DATA_PATH'], 'ml-latest-small')
data_path = os.environ['DATA_PATH']
data_path

'E:\\MYLEARN\\2-ANALYTICS-DataScience\\datasets'

In [3]:
# small
# movies_filename  = 'small-movies.csv'
# ratings_filename = 'small-ratings.csv'

# 1 million
movies_filename  = '1m-movies.dat'
ratings_filename = '1m-ratings.dat'
users_filename   = '1m-users.dat'

# 10 million
# movies_filename  = '10m-movies.dat'
# ratings_filename = '10m-ratings.dat'


In [4]:
df_movies   = pd.read_csv(os.path.join(data_path, movies_filename), 
                          names=['movie_id', 'title', 'genre'], 
                          sep='::', 
                          engine='python')

df_ratings  = pd.read_csv(os.path.join(data_path, ratings_filename), 
                          names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                          sep='::', 
                          engine='python')

df_users    = pd.read_csv(os.path.join(data_path, users_filename), 
                          names=['user_id', 'gender', 'age', 'occupation', 'zipcode'], 
                          sep='::', 
                          engine='python')


In [6]:
print(df_movies.info())
print(df_ratings.info())

print(df_users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
movie_id    3883 non-null int64
title       3883 non-null object
genre       3883 non-null object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
user_id      1000209 non-null int64
movie_id     1000209 non-null int64
rating       1000209 non-null int64
timestamp    1000209 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
user_id       6040 non-null int64
gender        6040 non-null object
age           6040 non-null int64
occupation    6040 non-null int64
zipcode       6040 non-null object
dtypes: int64(3), object(2)
memory usage: 236.0+ KB
None


In [7]:
df_movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
df_ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
num_users   = len(df_ratings.user_id.unique())
num_items   = len(df_ratings.movie_id.unique())
num_ratings = len(df_ratings)

print('There are {} unique users, {} unique movies and {} ratings, in this data set'.format(num_users, num_items, num_ratings))

There are 6040 unique users, 3706 unique movies and 1000209 ratings, in this data set


In [10]:
df_users.head()

,user_id,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


# Exploratory data analysis

Plot the counts of each rating

Plot rating frequency of each movie

### 1. rating distribution

In [11]:
df_ratings.isnull().sum()

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

#### 1. Reshaping the Data

For K-Nearest Neighbors, we want the data to be in an (artist, user) array, where 
- one row per user and 
- one column per movie.

To reshape the dataframe, we'll pivot the dataframe to the wide format with movies as rows and users as columns. 


In [15]:
R_df = df_ratings.pivot(index   = 'user_id', 
                        columns = 'movie_id', 
                        values  = 'rating').fillna(0)
R_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
R_df.shape

(6040, 3706)

In [17]:
R = R_df.as_matrix()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [18]:
R

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]])

In [19]:
user_ratings_mean = np.mean(R, axis = 1)
user_ratings_mean

array([0.05990286, 0.12924987, 0.05369671, ..., 0.02050729, 0.1287102 ,
       0.3291959 ])

In [21]:
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[ 4.94009714, -0.05990286, -0.05990286, ..., -0.05990286,
        -0.05990286, -0.05990286],
       [-0.12924987, -0.12924987, -0.12924987, ..., -0.12924987,
        -0.12924987, -0.12924987],
       [-0.05369671, -0.05369671, -0.05369671, ..., -0.05369671,
        -0.05369671, -0.05369671],
       ...,
       [-0.02050729, -0.02050729, -0.02050729, ..., -0.02050729,
        -0.02050729, -0.02050729],
       [-0.1287102 , -0.1287102 , -0.1287102 , ..., -0.1287102 ,
        -0.1287102 , -0.1287102 ],
       [ 2.6708041 , -0.3291959 , -0.3291959 , ..., -0.3291959 ,
        -0.3291959 , -0.3291959 ]])

# Singular Value Decomposition
Scipy and Numpy both have functions to do the singular value decomposition.

In [22]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(R_demeaned, k = 50)

In [23]:
U

array([[ 4.97801875e-03,  5.86971868e-03, -1.18186843e-02, ...,
        -2.95139320e-03, -1.95703358e-03, -5.46889776e-03],
       [ 1.10526375e-03, -4.04545890e-03,  1.16776791e-02, ...,
        -9.18855171e-04,  2.17034433e-03, -1.04359614e-02],
       [-9.44963839e-03, -1.43519545e-02, -3.93638119e-04, ...,
         2.89764529e-03,  2.86504507e-03, -6.13985002e-03],
       ...,
       [ 1.05731072e-02, -6.80807641e-03, -2.63392883e-03, ...,
         4.84286245e-05, -1.89077440e-03, -1.52456048e-03],
       [-6.34420788e-03, -9.45269844e-03,  2.69929558e-03, ...,
         1.07208981e-02, -1.88878158e-02, -6.87143535e-03],
       [ 1.84854679e-02,  1.28388950e-02,  8.46988257e-03, ...,
         1.89987575e-03, -4.15563933e-02, -1.92850979e-02]])

In [24]:
sigma

array([ 147.18581225,  147.62154312,  148.58855276,  150.03171353,
        151.79983807,  153.96248652,  154.29956787,  154.54519202,
        156.1600638 ,  157.59909505,  158.55444246,  159.49830789,
        161.17474208,  161.91263179,  164.2500819 ,  166.36342107,
        166.65755956,  167.57534795,  169.76284423,  171.74044056,
        176.69147709,  179.09436104,  181.81118789,  184.17680849,
        186.29341046,  192.15335604,  192.56979125,  199.83346621,
        201.19198515,  209.67692339,  212.55518526,  215.46630906,
        221.6502159 ,  231.38108343,  239.08619469,  244.8772772 ,
        252.13622776,  256.26466285,  275.38648118,  287.89180228,
        315.0835415 ,  335.08085421,  345.17197178,  362.26793969,
        415.93557804,  434.97695433,  497.2191638 ,  574.46932602,
        670.41536276, 1544.10679346])

In [26]:
Vt

array([[ 0.07028629, -0.02415349,  0.01883837, ..., -0.00380736,
         0.00049127, -0.00061123],
       [ 0.03681506,  0.00346263, -0.01264234, ..., -0.00965995,
        -0.00513455, -0.02377963],
       [ 0.03495646,  0.00904907,  0.00823098, ...,  0.00157338,
        -0.00234513,  0.00802561],
       ...,
       [-0.03287652,  0.01185799, -0.01107445, ..., -0.00114772,
        -0.00294575, -0.02222119],
       [ 0.01776333,  0.03068092,  0.01786526, ..., -0.00087071,
        -0.0012666 , -0.00435186],
       [-0.07625855, -0.01650222, -0.00468327, ...,  0.00852744,
         0.01020778, -0.00425656]])

In [27]:
sigma = np.diag(sigma)

In [28]:
sigma

array([[ 147.18581225,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,  147.62154312,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,  148.58855276, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,  574.46932602,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
         670.41536276,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        , 1544.10679346]])

# Making Predictions from the Decomposed Matrices

- movie ratings predictions for every user



In [35]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [36]:
all_user_predicted_ratings

array([[ 4.28886061,  0.14305516, -0.1950795 , ...,  0.03191195,
         0.05044975,  0.08891033],
       [ 0.74471587,  0.16965927,  0.33541808, ..., -0.10110207,
        -0.0540982 , -0.14018846],
       [ 1.81882382,  0.45613623,  0.09097801, ...,  0.01234452,
         0.01514752, -0.10995596],
       ...,
       [ 0.61908871, -0.16176859,  0.10673806, ..., -0.01336948,
        -0.0303543 , -0.11493552],
       [ 1.50360483, -0.03620761, -0.16126817, ..., -0.01090407,
        -0.03864749, -0.16835943],
       [ 1.99624816, -0.18598715, -0.1564782 , ..., -0.00664061,
         0.12706713,  0.28500112]])

In [38]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
preds_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,...,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,...,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,...,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,...,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,...,0.110151,0.046010,0.006934,-0.015940,-0.050080,-0.052539,0.507189,0.033830,0.125706,0.199244


In [45]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number         = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.user_id == (userID)]
    
    user_full = (user_data.merge(movies_df,
                                 how      = 'left', 
                                 left_on  = 'movie_id', 
                                 right_on = 'movie_id').
                                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [46]:
already_rated, predictions = recommend_movies(preds_df, 837, df_movies, df_ratings, 10)

User 837 has already rated 69 movies.
Recommending highest 10 predicted ratings movies not already rated.


In [47]:
predictions

,movie_id,title,genre
516,527,Schindler's List (1993),Drama|War
1848,1953,"French Connection, The (1971)",Action|Crime|Drama|Thriller
596,608,Fargo (1996),Crime|Drama|Thriller
1235,1284,"Big Sleep, The (1946)",Film-Noir|Mystery
2085,2194,"Untouchables, The (1987)",Action|Crime|Drama
1188,1230,Annie Hall (1977),Comedy|Romance
1198,1242,Glory (1989),Action|Drama|War
897,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir
1849,1954,Rocky (1976),Action|Drama
581,593,"Silence of the Lambs, The (1991)",Drama|Thriller
